# Getting Data Ready

The overall process for using Amazon Forecast is the following:

1. Create a Dataset Group, this is the large box that isolates models and the data they are trained on from each other.
1. Create a Dataset, in Forecast there are 3 types of dataset, Target Time Series, Related Time Series, and Item Metadata. The Target Time Series is required, the others provide additional context with certain algorithms. 
1. Import data, this moves the information from S3 into a storage volume where the data can be used for training and validation.
1. Train a model, Forecast automates this process for you but you can also select particular algorithms, and you can provide your own hyper parameters or use Hyper Parameter Optimization(HPO) to determine the most performant values for you.
1. Deploy a Predictor, here you are deploying your model so you can use it to generate a forecast.
1. Query the Predictor, given a request bounded by time for an item, return the forecast for it. Once you have this you can evaluate its performance or use it to guide your decisions about the future.

In this notebook we will be walking through the first 3 steps outlined above. One additional task that will be done here is to trim part of our training and validation data so that we can measure the accuracy of a forecast against our predictions. 


## Table Of Contents
* [Setting up](#setup)
* [Data Preparation](#DataPrep)
* [Creating the Dataset Group and Dataset](#dataset)
* [Forecasting Example with Amazon Forecast](#forecastingExample)


**Read Every Cell FULLY before executing it**

For more informations about APIs, please check the [documentation](https://docs.aws.amazon.com/forecast/latest/dg/what-is-forecast.html)

## Set up Preview SDK<a class="anchor" id="setup"></a>

Amazon Forecast is still in preview, to update to the latest functionality execute the cells below.

In [1]:
# Configures your AWS CLI for Amazon Forecast
!aws configure add-model --service-model file://../sdk/forecastquery-2019-07-22.normal.json --service-name forecastquery
!aws configure add-model --service-model file://../sdk/forecast-2019-07-22.normal.json --service-name forecast

Next import the standard Python Libraries that are used in this lesson.

In [4]:
import boto3
from time import sleep
import subprocess
import pandas as pd

The last part of the setup process is to validate that your account can communicate with Amazon Forecast, the cell below does just that.

In [3]:
session = boto3.Session(region_name='us-west-2') 
forecast = session.client(service_name='forecast') 
forecastquery = session.client(service_name='forecastquery')

## Data Preparation<a class="anchor" id="DataPrep"></a>

For this exercise, we use the individual household electric power consumption dataset. (Dua, D. and Karra Taniskidou, E. (2017). UCI Machine Learning Repository [http://archive.ics.uci.edu/ml]. Irvine, CA: University of California, School of Information and Computer Science.) We aggregate the usage data hourly. 

To begin, use Pandas to read the CSV and to show a sample of the data.

In [9]:
df = pd.read_csv("../data/item-demand-time.csv", dtype = object, names=['timestamp','value','item'])
df.head(3)

,timestamp,value,item
0,2014-01-01 01:00:00,38.34991708126038,client_12
1,2014-01-01 02:00:00,33.5820895522388,client_12
2,2014-01-01 03:00:00,34.41127694859037,client_12


Notice in the output above there are 3 columns of data:

1. The Timestamp
1. A Value
1. An Item

These are the 3 key required pieces of information to generate a forecast with Amazon Forecast. More can be added but these 3 must always remain present.

The dataset happens to span January 01, 2014 to Deceber 31, 2014. For our testing we would like to keep the last month of information in a differennt CSV. We are also going to save January to November to a different CSV as well. Both will be uploaded to S3 for use later.

You may notice a variable named `df` this is a popular convention when using Pandas if you are using the library's dataframe object, it is similar to a table in a database. You can learn more here: https://pandas.pydata.org/pandas-docs/stable/getting_started/10min.html


In [10]:
# Select January to November for one dataframe.
jan_to_nov = df[(df['timestamp'] >= '2014-01-01') & (df['timestamp'] <= '2014-11-30')]

# Select the month of December for another dataframe.
dec_df = df[(df['timestamp'] >= '2014-12-01') & (df['timestamp'] <= '2014-12-31')]

Now export them to CSV files and place them into your `data` folder.

In [12]:
jan_to_nov.to_csv("../data/item-demand-time.csv", header=False, index=False)
jan_to_nov.to_csv("../data/item-demand-time-validation.csv", header=False, index=False)

At this time the data is ready to be sent to S3 where Forecast will use it later. Update the bucketname in the cell below to reflect the value you obtained from our output earlier when you finished the CloudFormation step.

In [13]:
bucket_name = "forecastdemochrisking" # Rember to change this.

The following cells will upload the data to S3.

In [16]:
s3 = session.client('s3')
key="elec_data/item-demand-time.csv"
s3.upload_file(Filename="../data/item-demand-time.csv", Bucket=bucket_name, Key=key)

## Creating the Dataset Group and Dataset <a class="anchor" id="dataset"></a>

In Amazon Forecast , a dataset is a collection of file(s) which contain data that is relevant for a forecasting task. A dataset must conform to a schema provided by Amazon Forecast. 

More details about `Domain` and dataset type can be found on the [documentation](https://docs.aws.amazon.com/forecast/latest/dg/howitworks-domains-ds-types.html) . For this example, we are using [CUSTOM](https://docs.aws.amazon.com/forecast/latest/dg/custom-domain.html) domain with 3 required attributes `timestamp`, `target_value` and `item_id`.


It is importan to also convey how Amazon Forecast can understand your time-series information. That the cell immediately below does that, the next one configures your variable names for the Project, DatasetGroup, and Dataset.

In [17]:
DATASET_FREQUENCY = "H" 
TIMESTAMP_FORMAT = "yyyy-MM-dd hh:mm:ss"

In [24]:
project = 'power_forecastdemo'
datasetName= project+'_ds'
datasetGroupName= project +'_dsg'
s3DataPath = "s3://"+bucket_name+"/"+key

### Creating a Schema

Amazon Forecast relies on a concept called a schema to map the content in your CSV file into a format that Forecast can understand. The cells below will create a schema that matches the CSV provided, adn will call the API calls needed to build the Datastet and Datatset Group.

In [25]:
# Specify the schema of your dataset here. Make sure the order of columns matches the raw data files.
schema ={
   "Attributes":[
      {
         "AttributeName":"timestamp",
         "AttributeType":"timestamp"
      },
      {
         "AttributeName":"target_value",
         "AttributeType":"float"
      },
      {
         "AttributeName":"item_id",
         "AttributeType":"string"
      }
   ]
}

In [26]:
response=forecast.create_dataset(
                    Domain="CUSTOM",
                    DatasetType='TARGET_TIME_SERIES',
                    DatasetName=datasetName,
                    DataFrequency=DATASET_FREQUENCY, 
                    Schema = schema
)

In [27]:
datasetArn = response['DatasetArn']
forecast.describe_dataset(DatasetArn=datasetArn)

{'DatasetArn': 'arn:aws:forecast:us-west-2:059124553121:dataset/power_forecastdemo_ds',
 'DatasetName': 'power_forecastdemo_ds',
 'Domain': 'CUSTOM',
 'DatasetType': 'TARGET_TIME_SERIES',
 'DataFrequency': 'H',
 'Schema': {'Attributes': [{'AttributeName': 'timestamp',
    'AttributeType': 'timestamp'},
   {'AttributeName': 'target_value', 'AttributeType': 'float'},
   {'AttributeName': 'item_id', 'AttributeType': 'string'}]},
 'EncryptionConfig': {},
 'Status': 'ACTIVE',
 'CreationTime': datetime.datetime(2019, 8, 17, 21, 26, 5, 308000, tzinfo=tzlocal()),
 'LastModificationTime': datetime.datetime(2019, 8, 17, 21, 26, 5, 308000, tzinfo=tzlocal()),
 'ResponseMetadata': {'RequestId': 'ca6c25c1-7b3b-4df4-9508-dc09c1bc8e72',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Sat, 17 Aug 2019 21:26:17 GMT',
   'x-amzn-requestid': 'ca6c25c1-7b3b-4df4-9508-dc09c1bc8e72',
   'content-length': '503',
   'connection': 'keep-alive'},
  'RetryAttem

In [33]:
create_dataset_group_response = forecast.create_dataset_group(DatasetGroupName=datasetGroupName,
                                                              Domain="CUSTOM",
                                                              DatasetArns= [datasetArn]
                                                             )
datasetGroupArn = create_dataset_group_response['DatasetGroupArn']

ParamValidationError: Parameter validation failed:
Unknown parameter in input: "Domain", must be one of: DatasetGroupName, DatasetArns